In [ ]:
import quasar

In [ ]:
circuit = quasar.Circuit()
circuit.H(0).X(1).Y(2).Z(3)
circuit.CX(0,1).CX(2,3)
circuit.CX(0,3).CX(2,1)
circuit.CX(0,2).CX(3,1)
gadget = quasar.Circuit().Ry(0).CX(0,1)
circuit.add_gate(gadget, (0,1), ascii_symbols=['A0', 'A1'])
circuit.add_gate(gadget, (3,2), ascii_symbols=['B0', 'B1'])
circuit.add_gate(gadget, (0,3), ascii_symbols=['A0', 'A1'])
circuit.add_gate(gadget, (2,1), ascii_symbols=['B0', 'B1'])
circuit.add_gate(gadget, (0,2), ascii_symbols=['A0', 'A1'])
circuit.add_gate(gadget, (3,1), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,1), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,3,2), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,3), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,2,1), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,2), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,3,1), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,2), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,3,1), ascii_symbols=['B0', 'B1'])
circuit.add_controlled_gate(gadget, (4,0,2), ascii_symbols=['A0', 'A1'])
circuit.add_controlled_gate(gadget, (5,3,1), ascii_symbols=['B0', 'B1'])
import time
start = time.time()
print(circuit)
print('%11.3E' % (time.time() - start))

In [ ]:
print(quasar.Circuit().X(0, times=-1).X(0).X(0).X(0).X(0).X(0).X(0, time_start=10).X(0).X(0).X(0).X(0).X(0).X(0).X(0))

In [ ]:
print(quasar.Circuit())